````C
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <math.h>
#include "esp_log.h"
#include "nvs_flash.h"
#include "esp_event.h"
#include "esp_netif.h"

#include "freertos/FreeRTOS.h"
#include "freertos/task.h"
#include "esp_system.h"
#include "esp_adc_cal.h"

#include "esp_http_client.h"
#include "connect_wifi.h"
#include "driver/i2c.h"

static const char *TAG = "HTTP_CLIENT";
char api_key[] = "EH18IFEEIV9JKOHH";

float T=0;

void send_data_to_thingspeak(void *pvParameters)
{

	char thingspeak_url[] = "https://api.thingspeak.com";
	char data[] = "/update?api_key=%s&field1=%.2f&field2=%.2f&field3=%.2f";
	char post_data[200];
	esp_err_t err;

	esp_http_client_config_t config = {
		.url = thingspeak_url,
		.method = HTTP_METHOD_GET,
	};
	esp_http_client_handle_t client = esp_http_client_init(&config);
	esp_http_client_set_header(client, "Content-Type", "application/x-www-form-urlencoded");
	while (1)
	{
		T+=1;
		vTaskDelay(5000 / portTICK_RATE_MS);
		strcpy(post_data, "");
		snprintf(post_data, sizeof(post_data), data, api_key, T, T, T);
		ESP_LOGI(TAG, "post = %s", post_data);
		esp_http_client_set_post_field(client, post_data, strlen(post_data));
		esp_http_client_set_url(client, post_data);
		err = esp_http_client_perform(client);

		if (err == ESP_OK)
		{
			int status_code = esp_http_client_get_status_code(client);
			if (status_code == 200)
			{
				ESP_LOGI(TAG, "Message sent Successfully");
			}
			else
			{
				ESP_LOGI(TAG, "Message sent Failed");				
				goto exit;
			}
		}
		else
		{
			ESP_LOGI(TAG, "Message sent Failed");
			goto exit;
		}
	}
exit:
	esp_http_client_cleanup(client);
	vTaskDelete(NULL);
}

void app_main(void)
{
	
	esp_err_t ret = nvs_flash_init();
	if (ret == ESP_ERR_NVS_NO_FREE_PAGES || ret == ESP_ERR_NVS_NEW_VERSION_FOUND)
	{
		ESP_ERROR_CHECK(nvs_flash_erase());
		ret = nvs_flash_init();
	}
	ESP_ERROR_CHECK(ret);
	
	connect_wifi();
	if (wifi_connect_status)
	{
	
		xTaskCreate(&send_data_to_thingspeak, "send_data_to_thingspeak", 8192, NULL, 6, NULL);
	}
}
````

````C
#include "driver/i2c.h"
#include "esp_log.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>
#include "lcd-i2c.h"

#include "freertos/FreeRTOS.h"
#include "freertos/queue.h"
#include "freertos/task.h"
#include "driver/gpio.h"
#include "esp_adc_cal.h"
#include "unistd.h"
#include <time.h>

#define Den1 27
#define Den2 26

float V0=4095, R1= 5000, R0=10000,B=3950,T0=298.15;
float V1, v1, R2, r, T;

static esp_adc_cal_characteristics_t adc1_chars;
static const char *TAG = "i2c-simple-example";
char buffer[15];

TaskHandle_t myTaskHandle = NULL;
TaskHandle_t myTaskHandle1 = NULL;
TaskHandle_t myTaskHandle2 = NULL;

QueueHandle_t queue;
static esp_err_t i2c_master_init(void)
{
    int i2c_master_port = I2C_NUM_0;

    i2c_config_t conf = {
        .mode = I2C_MODE_MASTER,
        .sda_io_num = GPIO_NUM_21,
        .scl_io_num = GPIO_NUM_22,
        .sda_pullup_en = GPIO_PULLUP_ENABLE,
        .scl_pullup_en = GPIO_PULLUP_ENABLE,
        .master.clk_speed = 100000,
    };

    i2c_param_config(i2c_master_port, &conf);

    return i2c_driver_install(i2c_master_port, conf.mode, 0, 0, 0);
}
void Do(void *arg)
{
  int a;
    
    while(1)
    {
    
        V1 = adc1_get_raw(ADC1_CHANNEL_4); //lấy giá trị adc
        v1 = esp_adc_cal_raw_to_voltage(V1, &adc1_chars); //đọc giá trị điện áp từ giá trị adc
        R2=R1*(V0/V1-1);
        T=B/log(R2/r)-273.15;
         if(T<20)
           {
             a=1;
             xQueueSend(queue, &a, (TickType_t)0);
             xQueueSend(queue, &a, (TickType_t)0); // ghi dữ liệu vào hàng đợi
            }
            else if(T>=20 && T<=24)
            {
             a=2;
             xQueueSend(queue, &a, (TickType_t)0);
             xQueueSend(queue, &a, (TickType_t)0);
        
            }     
            else
            {
              a=3;
             xQueueSend(queue, &a, (TickType_t)0);
             xQueueSend(queue, &a, (TickType_t)0);
            }  
        vTaskDelay(1000/ portTICK_RATE_MS);
    
    }
}

void Nhay1(void *arg)
{
  int b; //tạo kí tự để lưu trữ dữ liệu sẽ nhận từ hàng đợi
  while(1)
  {
     xQueueReceive(queue, &b, (TickType_t)1000); //đọc dữ liệu từ hàng đợi
    if(b==1)
    { 
      printf("Received data from queue == %d, ", b);
      gpio_set_level(Den1, 1);
      vTaskDelay(500/ portTICK_RATE_MS);
      gpio_set_level(Den1, 0);
      vTaskDelay(500/ portTICK_RATE_MS);
    }
    else if(b==2)
    {
      printf("Received data from queue == %d, ", b);
      gpio_set_level(Den1, 1);
      vTaskDelay(500/ portTICK_RATE_MS);
      gpio_set_level(Den1, 0);
      vTaskDelay(500/ portTICK_RATE_MS);
    
    }
    else if(b==3)
    {
      printf("Received data from queue == %d, ", b);
      gpio_set_level(Den1, 0);
      
    }
  }
}
void Nhay2(void *arg)
{
  int c; //tạo kí tự để lưu trữ dữ liệu sẽ nhận từ hàng đợi
  while(1)
  {
     xQueueReceive(queue, &c, (TickType_t)1000); //đọc dữ liệu từ hàng đợi
    if(c==1)
    {
      printf("T1= %f, V1=%f, v1=%f mV, R2=%f\n", T, V1, v1, R2);
      gpio_set_level(Den2, 0);
      vTaskDelay(500/ portTICK_RATE_MS);
    }
    else if(c==2)
    {
      printf("T2= %f, V1=%f, v1=%f mV, R2=%f\n", T, V1, v1, R2);
      gpio_set_level(Den2, 1);
      vTaskDelay(250/ portTICK_RATE_MS);
      gpio_set_level(Den2, 0);
      vTaskDelay(250/ portTICK_RATE_MS);
      gpio_set_level(Den2, 1);
      vTaskDelay(250/ portTICK_RATE_MS);
      gpio_set_level(Den2, 0);
      vTaskDelay(250/ portTICK_RATE_MS);
    
    }
    else if(c==3)
    {
      printf("T3= %f, V1=%f, v1=%f mV, R2=%f\n", T, V1, v1, R2);
      gpio_set_level(Den2, 1);
      vTaskDelay(125/ portTICK_RATE_MS);
      gpio_set_level(Den2, 0);
      vTaskDelay(125/ portTICK_RATE_MS);
      gpio_set_level(Den2, 1);
      vTaskDelay(125/ portTICK_RATE_MS);
      gpio_set_level(Den2, 0);
      vTaskDelay(125/ portTICK_RATE_MS);
      gpio_set_level(Den2, 1);
      vTaskDelay(125/ portTICK_RATE_MS);
      gpio_set_level(Den2, 0);
      vTaskDelay(125/ portTICK_RATE_MS);
      gpio_set_level(Den2, 1);
      vTaskDelay(125/ portTICK_RATE_MS);
      gpio_set_level(Den2, 0);
      vTaskDelay(125/ portTICK_RATE_MS);
    }
  }
}
  

void app_main(void)
{
  gpio_pad_select_gpio(Den1); //đặt cấu hình Den1 làm GPIO
  gpio_pad_select_gpio(Den2); 
  gpio_set_direction(Den1, GPIO_MODE_OUTPUT);// đặt Den1 làm đầu ra
  gpio_set_direction(Den2, GPIO_MODE_OUTPUT);
  esp_adc_cal_characterize(ADC_UNIT_1, ADC_ATTEN_DB_11, ADC_WIDTH_BIT_DEFAULT, 0, &adc1_chars); //hiệu chỉnh ADC1 ở mức suy giảm 11db

  adc1_config_width(ADC_WIDTH_BIT_DEFAULT); // đặt cấu hình ADC1 ở độ rộng bit mặc định (12bit)
  adc1_config_channel_atten(ADC1_CHANNEL_4, ADC_ATTEN_DB_11);// đặt tham số suy giảm của ADC1 kênh 4 là GPIO32 thành 11db
  r=R0*exp(-B/T0); 
  
  queue = xQueueCreate(5, sizeof(int));// tạo hàng đợi

  xTaskCreate(Do, "Do", 2048, NULL, 10, &myTaskHandle);
  xTaskCreatePinnedToCore(Nhay1, "Nhay1", 2048, NULL,10, &myTaskHandle1, 0);
  xTaskCreatePinnedToCore(Nhay2, "Nhay2", 2048, NULL,10, &myTaskHandle2, 1);
   ESP_ERROR_CHECK(i2c_master_init());
    ESP_LOGI(TAG, "I2C initialized successfully");

    lcd_init();
    lcd_clear();
    while(1){
// 	lcd_put_cur(0, 0);
//    lcd_send_string("Hello world!");

//    lcd_put_cur(1, 0);
//    lcd_send_string("from ESP32");

    sprintf(buffer, "T=%.6f", T);
    lcd_put_cur(1, 0);
    lcd_send_string(buffer);	
	vTaskDelay(1000/ portTICK_RATE_MS);
}
}
````

````C
#include "driver/i2c.h"
#include "esp_log.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>
#include "lcd-i2c.h"

#include "freertos/FreeRTOS.h"
#include "freertos/queue.h"
#include "freertos/task.h"
#include "driver/gpio.h"
#include "esp_adc_cal.h"
#include "unistd.h"
#include <time.h>

#define Den1 27
#define Den2 26

float V0=4095, R1= 5000, R0=10000,B=3950,T0=298.15;
float V1, v1, R2, r, T, DObh;

static esp_adc_cal_characteristics_t adc1_chars;
static const char *TAG = "i2c-simple-example";
char buffer[15];

TaskHandle_t myTaskHandle = NULL;
TaskHandle_t myTaskHandle1 = NULL;
TaskHandle_t myTaskHandle2 = NULL;

QueueHandle_t queue;
static esp_err_t i2c_master_init(void)
{
    int i2c_master_port = I2C_NUM_0;

    i2c_config_t conf = {
        .mode = I2C_MODE_MASTER,
        .sda_io_num = GPIO_NUM_21,
        .scl_io_num = GPIO_NUM_22,
        .sda_pullup_en = GPIO_PULLUP_ENABLE,
        .scl_pullup_en = GPIO_PULLUP_ENABLE,
        .master.clk_speed = 100000,
    };

    i2c_param_config(i2c_master_port, &conf);

    return i2c_driver_install(i2c_master_port, conf.mode, 0, 0, 0);
}
void Do(void *arg)
{
  int a;
    
    while(1)
    {
    
        V1 = adc1_get_raw(ADC1_CHANNEL_4); //lấy giá trị adc
        v1 = esp_adc_cal_raw_to_voltage(V1, &adc1_chars); //đọc giá trị điện áp từ giá trị adc
        R2=R1*(V0/V1-1);
        T=B/log(R2/r)-273.15;
        DObh=14.652-0.41022*T +0.007991*pow(T, 2) - 0.000077774*pow(T, 3);

         if(T<20)
           {
             a=1;
             xQueueSend(queue, &a, (TickType_t)0);
             xQueueSend(queue, &a, (TickType_t)0); // ghi dữ liệu vào hàng đợi
            }
            else if(T>=20 && T<=24)
            {
             a=2;
             xQueueSend(queue, &a, (TickType_t)0);
             xQueueSend(queue, &a, (TickType_t)0);
        
            }     
            else
            {
              a=3;
             xQueueSend(queue, &a, (TickType_t)0);
             xQueueSend(queue, &a, (TickType_t)0);
            }  
        vTaskDelay(1000/ portTICK_RATE_MS);
    
    }
}

void Nhay1(void *arg)
{
  int b; //tạo kí tự để lưu trữ dữ liệu sẽ nhận từ hàng đợi
  while(1)
  {
     xQueueReceive(queue, &b, (TickType_t)1000); //đọc dữ liệu từ hàng đợi
    if(b==1)
    { 
      printf("Received data from queue == %d, ", b);
      gpio_set_level(Den1, 1);
      vTaskDelay(500/ portTICK_RATE_MS);
      gpio_set_level(Den1, 0);
      vTaskDelay(500/ portTICK_RATE_MS);
    }
    else if(b==2)
    {
      printf("Received data from queue == %d, ", b);
      gpio_set_level(Den1, 1);
      vTaskDelay(500/ portTICK_RATE_MS);
      gpio_set_level(Den1, 0);
      vTaskDelay(500/ portTICK_RATE_MS);
    
    }
    else if(b==3)
    {
      printf("Received data from queue == %d, ", b);
      gpio_set_level(Den1, 0);
      
    }
  }
}
void Nhay2(void *arg)
{
  int c; //tạo kí tự để lưu trữ dữ liệu sẽ nhận từ hàng đợi
  while(1)
  {
     xQueueReceive(queue, &c, (TickType_t)1000); //đọc dữ liệu từ hàng đợi
    if(c==1)
    {
      printf("T1= %f, V1=%f, v1=%f mV, R2=%f\n", T, V1, v1, R2);
      gpio_set_level(Den2, 0);
      vTaskDelay(500/ portTICK_RATE_MS);
    }
    else if(c==2)
    {
      printf("T2= %f, V1=%f, v1=%f mV, R2=%f\n", T, V1, v1, R2);
      gpio_set_level(Den2, 1);
      vTaskDelay(250/ portTICK_RATE_MS);
      gpio_set_level(Den2, 0);
      vTaskDelay(250/ portTICK_RATE_MS);
      gpio_set_level(Den2, 1);
      vTaskDelay(250/ portTICK_RATE_MS);
      gpio_set_level(Den2, 0);
      vTaskDelay(250/ portTICK_RATE_MS);
    
    }
    else if(c==3)
    {
      printf("T3= %f, V1=%f, v1=%f mV, R2=%f\n", T, V1, v1, R2);
      gpio_set_level(Den2, 1);
      vTaskDelay(125/ portTICK_RATE_MS);
      gpio_set_level(Den2, 0);
      vTaskDelay(125/ portTICK_RATE_MS);
      gpio_set_level(Den2, 1);
      vTaskDelay(125/ portTICK_RATE_MS);
      gpio_set_level(Den2, 0);
      vTaskDelay(125/ portTICK_RATE_MS);
      gpio_set_level(Den2, 1);
      vTaskDelay(125/ portTICK_RATE_MS);
      gpio_set_level(Den2, 0);
      vTaskDelay(125/ portTICK_RATE_MS);
      gpio_set_level(Den2, 1);
      vTaskDelay(125/ portTICK_RATE_MS);
      gpio_set_level(Den2, 0);
      vTaskDelay(125/ portTICK_RATE_MS);
    }
  }
}

void app_main(void)
{
  gpio_pad_select_gpio(Den1); //đặt cấu hình Den1 làm GPIO
  gpio_pad_select_gpio(Den2); 
  gpio_set_direction(Den1, GPIO_MODE_OUTPUT);// đặt Den1 làm đầu ra
  gpio_set_direction(Den2, GPIO_MODE_OUTPUT);
  esp_adc_cal_characterize(ADC_UNIT_1, ADC_ATTEN_DB_11, ADC_WIDTH_BIT_DEFAULT, 0, &adc1_chars); //hiệu chỉnh ADC1 ở mức suy giảm 11db

  adc1_config_width(ADC_WIDTH_BIT_DEFAULT); // đặt cấu hình ADC1 ở độ rộng bit mặc định (12bit)
  adc1_config_channel_atten(ADC1_CHANNEL_4, ADC_ATTEN_DB_11);// đặt tham số suy giảm của ADC1 kênh 4 là GPIO32 thành 11db
  r=R0*exp(-B/T0); 
  
  queue = xQueueCreate(5, sizeof(int));// tạo hàng đợi

  xTaskCreate(Do, "Do", 2048, NULL, 10, &myTaskHandle);
  xTaskCreatePinnedToCore(Nhay1, "Nhay1", 2048, NULL,10, &myTaskHandle1, 0);
  xTaskCreatePinnedToCore(Nhay2, "Nhay2", 2048, NULL,10, &myTaskHandle2, 1);
   ESP_ERROR_CHECK(i2c_master_init());
    ESP_LOGI(TAG, "I2C initialized successfully");

    lcd_init();
    lcd_clear();
    while(1){
    sprintf(buffer, "DObh=%.6f", DObh);
    lcd_put_cur(1, 0);
    lcd_send_string(buffer);	
	vTaskDelay(1000/ portTICK_RATE_MS);
}
}
````

````C
#include "driver/i2c.h"
#include "esp_log.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>
#include "lcd-i2c.h"
#include <time.h>

#define SECONDS_MASK 0x7f
#define HOUR12_MASK  0x1f
#define HOUR24_MASK  0x3f
#define HOUR12_BIT  (1 << 6)
#define PM_BIT      (1 << 5)
#define DS1307_ADDR 0x68

static uint8_t bcd2dec(uint8_t val)
{
    return (val/16) * 10 + (val %16);
}

static uint8_t dec2bcd(uint8_t val)
{
    return ((val / 10) << 4) + (val % 10);
}

void ds1307_set_time(const struct tm *time)
{
    uint8_t buf[7] = {
        dec2bcd(time->tm_sec),
        dec2bcd(time->tm_min),
        dec2bcd(time->tm_hour),
        dec2bcd(time->tm_wday + 1),
        dec2bcd(time->tm_mday),
        dec2bcd(time->tm_mon + 1),
        dec2bcd(time->tm_year - 100)
    };
    i2c_cmd_handle_t cmd = i2c_cmd_link_create();
    i2c_master_start(cmd);
    i2c_master_write_byte(cmd, (DS1307_ADDR << 1) | I2C_MASTER_WRITE, true);
    i2c_master_write_byte(cmd, 0x00, true);
    i2c_master_write(cmd, buf, 7, true);
    i2c_master_stop(cmd);
    i2c_master_cmd_begin(I2C_NUM_0, cmd, 1000 / portTICK_RATE_MS);
    i2c_cmd_link_delete(cmd);

}

void ds1307_get_time(struct tm *time)
{
    uint8_t buf[7];
    i2c_cmd_handle_t cmd = i2c_cmd_link_create();
    i2c_master_start(cmd);
    i2c_master_write_byte(cmd, (DS1307_ADDR << 1) | I2C_MASTER_WRITE, true);
    i2c_master_write_byte(cmd, 0x00, true);
    i2c_master_start(cmd);
    i2c_master_write_byte(cmd, (DS1307_ADDR << 1) | I2C_MASTER_READ, true);
    i2c_master_read(cmd, buf, 7, I2C_MASTER_LAST_NACK);
    i2c_master_stop(cmd);
    i2c_master_cmd_begin(I2C_NUM_0, cmd, 1000 / portTICK_RATE_MS);
    i2c_cmd_link_delete(cmd);

    time->tm_sec = bcd2dec(buf[0] & SECONDS_MASK);
    time->tm_min = bcd2dec(buf[1]);
    if(buf[2] & HOUR12_BIT)
    {
        // RTC in 12-hour mode
        time->tm_hour = bcd2dec(buf[2] & HOUR12_MASK) - 1;
        if (buf[2] & PM_BIT)
            time->tm_hour += 12;
    }
    else
        time->tm_hour = bcd2dec(buf[2] & HOUR24_MASK);
    time->tm_wday = bcd2dec(buf[3]) - 1;
    time->tm_mday = bcd2dec(buf[4]);
    time->tm_mon  = bcd2dec(buf[5]) - 1;
    time->tm_year = bcd2dec(buf[6]) + 100;
}
const char* get_day_of_week(const struct tm *time)
{
    static const char* const days[] = {"Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"};
    return days[time->tm_wday];
}
void app_main() {
    i2c_config_t i2c_config = {
        .mode = I2C_MODE_MASTER,
        .sda_io_num = 21,
        .scl_io_num = 22,
        .sda_pullup_en = GPIO_PULLUP_ENABLE,
        .scl_pullup_en = GPIO_PULLUP_ENABLE,
        .master.clk_speed = 100000
    };
    i2c_param_config(I2C_NUM_0, &i2c_config);
    i2c_driver_install(I2C_NUM_0, I2C_MODE_MASTER, 0, 0, 0);

    struct tm time = {.tm_year = 123, .tm_mon = 04, .tm_mday = 17, .tm_wday=03, .tm_hour = 11, .tm_min =30, .tm_sec = 30};
    ds1307_set_time(&time);

    while (1) {
        ds1307_get_time(&time);
        const char* day_of_week = get_day_of_week(&time);
        printf("%02d:%02d:%02d %s %02d/%02d/%04d \n", time.tm_hour, time.tm_min, time.tm_sec, day_of_week,
                                                time.tm_mday,time.tm_mon + 1,time.tm_year + 1900);
        vTaskDelay(pdMS_TO_TICKS(1000));
    }
}

````C
#include "driver/i2c.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>

#include "freertos/FreeRTOS.h"
#include "freertos/queue.h"
#include "freertos/task.h"
#include "driver/gpio.h"
#include "esp_adc_cal.h"
#include "esp_system.h"

#include "esp_http_client.h"
#include "connect_wifi.h"
#include "esp_log.h"
#include "nvs_flash.h"
#include "esp_event.h"
#include "esp_netif.h"

static const char *TAG = "HTTP_CLIENT";
char api_key[] = "EH18IFEEIV9JKOHH";

#define SCOUNT  10
float T, DObh, ntu, tds ;

static esp_adc_cal_characteristics_t adc1_chars;
QueueHandle_t queue;


int getMedianNum(int bArray[], int iFilterLen){
  int bTab[iFilterLen];
  for (int i = 0; i<iFilterLen; i++){
    bTab[i] = bArray[i];// sao chép mảng 
  }
  int bTemp;
  for (int i = 0; i < iFilterLen - 1; i++){
    for (int j = 0; j < iFilterLen - i - 1; j++){ 
      if (bTab[j] > bTab[j + 1]){ //sắp xếp từ nhỏ đến lớn
        bTemp = bTab[j];
        bTab[j] = bTab[j + 1];
        bTab[j + 1] = bTemp;
      }
    }
  }
  if ((iFilterLen & 1) > 0){// nếu lẻ thì gán bằng phần tử chính giữa
      bTemp = bTab[(iFilterLen - 1) / 2];
  }
  else{
    bTemp = (bTab[iFilterLen / 2] + bTab[iFilterLen / 2 - 1]) / 2;
  }
  return bTemp;
}

float Array(char channel){
  int analogBuffer[SCOUNT];  
  int analogBufferTemp[SCOUNT];
  int analogBufferIndex = 0;
  float averageVoltage = 0;
  while(1){
  static unsigned long analogSampleTimepoint  = millis();
    //analogSampleTimepoint = esp_timer_get_time();
    if(esp_timer_get_time()-analogSampleTimepoint > 40U)     
    {
      analogSampleTimepoint = esp_timer_get_time();
      analogBuffer[analogBufferIndex] = adc1_get_raw(channel); 
      analogBufferIndex++;
      if(analogBufferIndex == SCOUNT){
        analogBufferIndex = 0;
      }
    }   
    static unsigned long printTimepoint = esp_timer_get_time();
    //printTimepoint = esp_timer_get_time();
    if(esp_timer_get_time()-printTimepoint > 800U)
    {
      printTimepoint = esp_timer_get_time();
      for(int copyIndex=0;copyIndex<SCOUNT;copyIndex++){
        analogBufferTemp[copyIndex]= analogBuffer[copyIndex];
      }
      averageVoltage = (float)getMedianNum(analogBufferTemp,SCOUNT)*5/4095;
      return averageVoltage;
    }
  }
}

void DO_bh(void *arg){
  float V0=4095, R1= 5000, R0=10000,B=3950,T0=298.15;
  float V1, R2, r;
  r=R0*exp(-B/T0);
  int a=1;
  while(1)
  {     
        V1 = Array(ADC1_CHANNEL_4);
        R2=R1*(V0/V1-1);
        T=B/log(R2/r)-273.15;
        xQueueSend(queue, &a, (TickType_t)0);
        DObh=14.652-0.41022*T +0.007991*pow(T, 2) - 0.000077774*pow(T, 3);
        printf("T=%f, DObh=%f\n ", T, DObh);
        vTaskDelay(1000/ portTICK_RATE_MS);
  }
}

void NTU(void *arg){
	float V0=5;
  float V1, V2;
  while (1){
    V1= (float)Array(ADC1_CHANNEL_5)*V0/4095;
	V2= V0-V1;
    if(V2 < 2.5){
      ntu = 3000;
    }else{
      ntu = -1120.4*pow(V2,2)+5742.3*V2-4353.8; 
    }
    printf("ntu=%f\n", ntu);
    vTaskDelay(1000/ portTICK_RATE_MS);
  }
}

void TDS(void *arg){
  float V0=5;
  float V3;
  int b;
  xQueueReceive(queue, &b, (TickType_t)1000);
  while (b==1){
    V3= (float)Array(ADC1_CHANNEL_6)*V0/4095;
    float compensationCoefficient=1.0+0.02*(T-25.0);  
    float compensationVolatge=V3/compensationCoefficient; 
    tds=(133.42*pow(compensationVolatge,3) - 255.86*pow(compensationVolatge,2) + 857.39*compensationVolatge)*0.5;
    printf("tds=%f\n ", tds);
    vTaskDelay(1000/ portTICK_RATE_MS);
  }
}

void send_data_to_thingspeak(void *pvParameters)
{

	char thingspeak_url[] = "https://api.thingspeak.com";
	char data[] = "/update?api_key=%s&field1=%.2f&field2=%.2f&field3=%.2f&field4=%.2f";
	char post_data[200];
	esp_err_t err;

	esp_http_client_config_t config = {
		.url = thingspeak_url,
		.method = HTTP_METHOD_GET,
	};
	esp_http_client_handle_t client = esp_http_client_init(&config);
	esp_http_client_set_header(client, "Content-Type", "application/x-www-form-urlencoded");
	while (1)
	{
		T+=1;
		vTaskDelay(2000 / portTICK_RATE_MS);
		strcpy(post_data, "");
		snprintf(post_data, sizeof(post_data), data, api_key, T, DObh, ntu, tds);
		ESP_LOGI(TAG, "post = %s", post_data);
		esp_http_client_set_post_field(client, post_data, strlen(post_data));
		esp_http_client_set_url(client, post_data);
		err = esp_http_client_perform(client);

		if (err == ESP_OK)
		{
			int status_code = esp_http_client_get_status_code(client);
			if (status_code == 200)
			{
				ESP_LOGI(TAG, "Message sent Successfully");
			}
			else
			{
				ESP_LOGI(TAG, "Message sent Failed");				
				goto exit;
			}
		}
		else
		{
			ESP_LOGI(TAG, "Message sent Failed");
			goto exit;
		}
	}
exit:
	esp_http_client_cleanup(client);
	vTaskDelete(NULL);
}
void app_main(void)
{
  esp_adc_cal_characterize(ADC_UNIT_1, ADC_ATTEN_DB_11, ADC_WIDTH_BIT_DEFAULT, 0, &adc1_chars); //hiệu chỉnh ADC1 ở mức suy giảm 11db
  adc1_config_width(ADC_WIDTH_BIT_DEFAULT); // đặt cấu hình ADC1 ở độ rộng bit mặc định (12bit)
  adc1_config_channel_atten(ADC1_CHANNEL_4, ADC_ATTEN_DB_11);// đặt tham số suy giảm của ADC1 kênh 4 là GPIO32 thành 11db
  adc1_config_channel_atten(ADC1_CHANNEL_5, ADC_ATTEN_DB_11);
  adc1_config_channel_atten(ADC1_CHANNEL_6, ADC_ATTEN_DB_11);
  queue = xQueueCreate(5, sizeof(int));
  
  xTaskCreatePinnedToCore(DO_bh, "DO_bh", 2048, NULL, 10, NULL,0);
  xTaskCreatePinnedToCore(NTU, "NTU", 2048, NULL, 10, NULL,1);
  xTaskCreatePinnedToCore(TDS, "TDS", 2048, NULL, 10, NULL,0);

  esp_err_t ret = nvs_flash_init();
	if (ret == ESP_ERR_NVS_NO_FREE_PAGES || ret == ESP_ERR_NVS_NEW_VERSION_FOUND)
	{
		ESP_ERROR_CHECK(nvs_flash_erase());
		ret = nvs_flash_init();
	}
	ESP_ERROR_CHECK(ret);
	
	connect_wifi();
	if (wifi_connect_status)
	{
	
		xTaskCreate(&send_data_to_thingspeak, "send_data_to_thingspeak", 8192, NULL, 6, NULL);
	}
}
````







````C
#include "driver/i2c.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>

#include "freertos/FreeRTOS.h"
#include "freertos/queue.h"
#include "freertos/task.h"
#include "driver/gpio.h"
#include "esp_adc_cal.h"
#include "esp_system.h"

#include "esp_http_client.h"
#include "connect_wifi.h"
#include "esp_log.h"
#include "nvs_flash.h"
#include "esp_event.h"
#include "esp_netif.h"

static const char *TAG = "HTTP_CLIENT";
char api_key[] = "EH18IFEEIV9JKOHH";

#define SCOUNT 30
float T, DObh, ntu, tds ;

static esp_adc_cal_characteristics_t adc1_chars;
QueueHandle_t queue;


int getMedianNum(int bArray[], int iFilterLen){
  int bTab[iFilterLen];
  for (int i = 0; i<iFilterLen; i++){
    bTab[i] = bArray[i];
  }
  int bTemp;
  for (int i = 0; i < iFilterLen - 1; i++){
    for (int j = 0; j < iFilterLen - i - 1; j++){ 
      if (bTab[j] > bTab[j + 1]){ 
        bTemp = bTab[j];
        bTab[j] = bTab[j + 1];
        bTab[j + 1] = bTemp;
      }
    }
  }
  if ((iFilterLen & 1) > 0){
      bTemp = bTab[(iFilterLen - 1) / 2];
  }
  else{
    bTemp = (bTab[iFilterLen / 2] + bTab[iFilterLen / 2 - 1]) / 2;
  }
  return bTemp;
}


void DO_bh(void *arg){
  float V0=4095, R1= 5000, R0=10000,B=3950,T0=298.15;
  float V1=0, R2, r;
  int analogBuffer[SCOUNT];  
  int analogBufferTemp[SCOUNT];
  int analogBufferIndex = 0;
  int a=1;
  r=R0*exp(-B/T0);
  while(1)
  {  
    analogBuffer[analogBufferIndex] = adc1_get_raw(ADC1_CHANNEL_4); 
    analogBufferIndex++; 
    vTaskDelay(40/ portTICK_RATE_MS);
    if(analogBufferIndex == SCOUNT)
    {
      analogBufferIndex = 0;
      for(int copyIndex=0;copyIndex<SCOUNT;copyIndex++){
        analogBufferTemp[copyIndex]= analogBuffer[copyIndex];
      }
      V1 = (float)getMedianNum(analogBufferTemp,SCOUNT);
      R2=R1*(V0/V1-1);
      T=B/log(R2/r)-273.15;
      xQueueSend(queue, &a, (TickType_t)0);
      DObh=14.652-0.41022*T +0.007991*pow(T, 2) - 0.000077774*pow(T, 3);
      printf("T=%f, DObh=%f\n ", T, DObh);
      vTaskDelay(1000/ portTICK_RATE_MS);
    }
  }
}

void NTU(void *arg){
	float V0=5;
  float V1, V2;
  int analogBuffer[SCOUNT];  
  int analogBufferTemp[SCOUNT];
  int analogBufferIndex = 0;
  while(1)
  {  
    analogBuffer[analogBufferIndex] = adc1_get_raw(ADC1_CHANNEL_5); 
    analogBufferIndex++;
    vTaskDelay(40/ portTICK_RATE_MS);
    if(analogBufferIndex == SCOUNT)
    {
      analogBufferIndex = 0;
      for(int copyIndex=0;copyIndex<SCOUNT;copyIndex++){
        analogBufferTemp[copyIndex]= analogBuffer[copyIndex];
      }
      V1 = (float)getMedianNum(analogBufferTemp,SCOUNT)*V0/4095;
	    V2= V0-V1;
      if(V2 < 2.5){
        ntu = 3000;
      }
      else{
        ntu = -1120.4*pow(V2,2)+5742.3*V2-4353.8; 
      }
      printf("ntu=%f\n", ntu);
      vTaskDelay(1000/ portTICK_RATE_MS);
    }
  }
}

void TDS(void *arg){
  float V0=5;
  float V3;
  int b;
  int analogBuffer[SCOUNT];  
  int analogBufferTemp[SCOUNT];
  int analogBufferIndex = 0;
  xQueueReceive(queue, &b, (TickType_t)1000);
  while (b==1){
    analogBuffer[analogBufferIndex] = adc1_get_raw(ADC1_CHANNEL_6); 
    analogBufferIndex++;
    vTaskDelay(40/ portTICK_RATE_MS);
    if(analogBufferIndex == SCOUNT)
    {
      analogBufferIndex = 0;
      for(int copyIndex=0;copyIndex<SCOUNT;copyIndex++){
        analogBufferTemp[copyIndex]= analogBuffer[copyIndex];
      }
      V3 = (float)getMedianNum(analogBufferTemp,SCOUNT)*V0/4095;
      float compensationCoefficient=1.0+0.02*(T-25.0);  
      float compensationVolatge=V3/compensationCoefficient; 
      tds=(133.42*pow(compensationVolatge,3) - 255.86*pow(compensationVolatge,2) + 857.39*compensationVolatge)*0.5;
      printf("tds=%f\n ", tds);
      vTaskDelay(1000/ portTICK_RATE_MS);
    }
  }
}

void send_data_to_thingspeak(void *pvParameters)
{

	char thingspeak_url[] = "https://api.thingspeak.com";
	char data[] = "/update?api_key=%s&field1=%.2f&field2=%.2f&field3=%.2f&field4=%.2f";
	char post_data[200];
	esp_err_t err;

	esp_http_client_config_t config = {
		.url = thingspeak_url,
		.method = HTTP_METHOD_GET,
	};
	esp_http_client_handle_t client = esp_http_client_init(&config);
	esp_http_client_set_header(client, "Content-Type", "application/x-www-form-urlencoded");
	while (1)
	{
		
		vTaskDelay(2000 / portTICK_RATE_MS);
		strcpy(post_data, "");
		snprintf(post_data, sizeof(post_data), data, api_key, T, DObh, ntu, tds);
		ESP_LOGI(TAG, "post = %s", post_data);
		esp_http_client_set_post_field(client, post_data, strlen(post_data));
		esp_http_client_set_url(client, post_data);
		err = esp_http_client_perform(client);

		if (err == ESP_OK)
		{
			int status_code = esp_http_client_get_status_code(client);
			if (status_code == 200)
			{
				ESP_LOGI(TAG, "Message sent Successfully");
			}
			else
			{
				ESP_LOGI(TAG, "Message sent Failed");				
				goto exit;
			}
		}
		else
		{
			ESP_LOGI(TAG, "Message sent Failed");
			goto exit;
		}
	}
exit:
	esp_http_client_cleanup(client);
	vTaskDelete(NULL);
}
void app_main(void)
{
  esp_adc_cal_characterize(ADC_UNIT_1, ADC_ATTEN_DB_11, ADC_WIDTH_BIT_DEFAULT, 0, &adc1_chars); //hiệu chỉnh ADC1 ở mức suy giảm 11db
  adc1_config_width(ADC_WIDTH_BIT_DEFAULT); // đặt cấu hình ADC1 ở độ rộng bit mặc định (12bit)
  adc1_config_channel_atten(ADC1_CHANNEL_4, ADC_ATTEN_DB_11);// đặt tham số suy giảm của ADC1 kênh 4 là GPIO32 thành 11db
  adc1_config_channel_atten(ADC1_CHANNEL_5, ADC_ATTEN_DB_11);
  adc1_config_channel_atten(ADC1_CHANNEL_6, ADC_ATTEN_DB_11);
  queue = xQueueCreate(5, sizeof(int));

  esp_err_t ret = nvs_flash_init();
	if (ret == ESP_ERR_NVS_NO_FREE_PAGES || ret == ESP_ERR_NVS_NEW_VERSION_FOUND)
	{
		ESP_ERROR_CHECK(nvs_flash_erase());
		ret = nvs_flash_init();
	}
	ESP_ERROR_CHECK(ret);
	
	connect_wifi();
	if (wifi_connect_status)
	{
    xTaskCreatePinnedToCore(DO_bh, "DO_bh", 2048, NULL, 10, NULL,0);
    xTaskCreatePinnedToCore(NTU, "NTU", 2048, NULL, 10, NULL,1);
    xTaskCreatePinnedToCore(TDS, "TDS", 2048, NULL, 10, NULL,0);
	
		xTaskCreate(&send_data_to_thingspeak, "send_data_to_thingspeak", 8192, NULL, 10, NULL);
	}
}



````C
#include <stdio.h>
#include <stdint.h>
#include <stddef.h>
#include <string.h>
#include "esp_wifi.h"
#include "esp_system.h"
#include "nvs_flash.h"
#include "esp_event.h"
#include "esp_netif.h"

#include "freertos/FreeRTOS.h"
#include "freertos/task.h"
#include "freertos/queue.h"

#include "lwip/sockets.h"
#include "lwip/dns.h"
#include "lwip/netdb.h"

#include "esp_log.h"
#include "mqtt_client.h"

#include "driver/i2c.h"
#include <math.h>
#include "lcd_i2c.h"
#include <DS_1307.h>

#include "driver/gpio.h"
#include "esp_adc_cal.h"
#include "unistd.h"
#include <time.h>

static const char *TAG_MQTT = "MQTT_EXAMPLE";
#define EXAMPLE_ESP_WIFI_SSID "Realme3proTQT"
#define EXAMPLE_ESP_WIFI_PASS "123456789"
#define MAX_RETRY 10
static int retry_cnt = 0;

#define THINGSPEAK_API_KEY "YOUR_THINGSPEAK_API_KEY"
#define THINGSPEAK_URI "mqtt://mqtt3.thingspeak.com:1883"
uint32_t MQTT_CONNECTED = 0;
static void mqtt_app_start(void);

#define Den1 27
#define Den2 26

float V0 = 4095, R1 = 5000, R0 = 10000, B = 3950, T0 = 298.15;
float V1, R2, r, T;

static esp_adc_cal_characteristics_t adc1_chars;
static const char *TAG = "i2c-simple-example";
char buffer0[15];
char buffer1[15];

QueueHandle_t queue;
static esp_err_t i2c_master_init(void)
{
    int i2c_master_port = I2C_NUM_0;

    i2c_config_t conf = {
        .mode = I2C_MODE_MASTER,
        .sda_io_num = GPIO_NUM_21,
        .scl_io_num = GPIO_NUM_22,
        .sda_pullup_en = GPIO_PULLUP_ENABLE,
        .scl_pullup_en = GPIO_PULLUP_ENABLE,
        .master.clk_speed = 100000,
    };

    i2c_param_config(i2c_master_port, &conf);

    return i2c_driver_install(i2c_master_port, conf.mode, 0, 0, 0);
}

static esp_err_t wifi_event_handler(void *arg, esp_event_base_t event_base,
                                    int32_t event_id, void *event_data)
{
    switch (event_id)
    {
    case WIFI_EVENT_STA_START:
        esp_wifi_connect();
        ESP_LOGI(TAG_MQTT, "Trying to connect with Wi-Fi\n");
        break;

    case WIFI_EVENT_STA_CONNECTED:
        ESP_LOGI(TAG_MQTT, "Wi-Fi connected\n");
        break;

    case IP_EVENT_STA_GOT_IP:
        ESP_LOGI(TAG_MQTT, "got ip: starting MQTT Client\n");
        mqtt_app_start();
        break;

    case WIFI_EVENT_STA_DISCONNECTED:
        ESP_LOGI(TAG_MQTT, "disconnected: Retrying Wi-Fi\n");
        if (retry_cnt++ < MAX_RETRY)
        {
            esp_wifi_connect();
        }
        else
            ESP_LOGI(TAG_MQTT, "Max Retry Failed: Wi-Fi Connection\n");
        break;

    default:
        break;
    }
    return ESP_OK;
}

void wifi_init(void)
{
    esp_event_loop_create_default();
    esp_event_handler_register(WIFI_EVENT, ESP_EVENT_ANY_ID, &wifi_event_handler, NULL);
    esp_event_handler_register(IP_EVENT, IP_EVENT_STA_GOT_IP, &wifi_event_handler, NULL);

    wifi_config_t wifi_config = {
        .sta = {
            .ssid = EXAMPLE_ESP_WIFI_SSID,
            .password = EXAMPLE_ESP_WIFI_PASS,
            .threshold.authmode = WIFI_AUTH_WPA2_PSK,
        },
    };
    esp_netif_init();
    esp_netif_create_default_wifi_sta();
    wifi_init_config_t cfg = WIFI_INIT_CONFIG_DEFAULT();
    esp_wifi_init(&cfg);
    esp_wifi_set_mode(WIFI_MODE_STA);
    esp_wifi_set_config(ESP_IF_WIFI_STA, &wifi_config);
    esp_wifi_start();
}

esp_mqtt_client_handle_t client;

static void mqtt_event_handler(void *handler_args, esp_event_base_t base, int32_t event_id, void *event_data)
{
    ESP_LOGD(TAG_MQTT, "Event dispatched from event loop base=%s, event_id=%d", base, event_id);
    esp_mqtt_event_handle_t event = event_data;
    client = event->client;
    int msg_id;
    switch ((esp_mqtt_event_id_t)event_id)
    {
    case MQTT_EVENT_CONNECTED:
        ESP_LOGI(TAG_MQTT, "MQTT_EVENT_CONNECTED");
        MQTT_CONNECTED = 1;
        break;
    case MQTT_EVENT_DISCONNECTED:
        ESP_LOGI(TAG_MQTT, "MQTT_EVENT_DISCONNECTED");
        MQTT_CONNECTED = 0;
        break;

    case MQTT_EVENT_ERROR:
        ESP_LOGI(TAG_MQTT, "MQTT_EVENT_ERROR");
        break;
    default:
        ESP_LOGI(TAG_MQTT, "Other event id:%d", event->event_id);
        break;
    }
}

static void mqtt_app_start(void)
{
    ESP_LOGI(TAG_MQTT, "STARTING MQTT");
    esp_mqtt_client_config_t mqttConfig = {
        .uri = THINGSPEAK_URI,
        .username = "ICwiKBYCMgUSFjMiHRgQPSE",
        .client_id = "ICwiKBYCMgUSFjMiHRgQPSE",
        .password = "enT4M0pNIfj6Dsu23yt5Cf7+",
    };

    client = esp_mqtt_client_init(&mqttConfig);
    esp_mqtt_client_register_event(client, ESP_EVENT_ANY_ID, mqtt_event_handler, client);
    esp_mqtt_client_start(client);
}

int getMedianNum(int bArray[], int iFilterLen)
{
    int bTab[iFilterLen];
    for (int i = 0; i < iFilterLen; i++)
    {
        bTab[i] = bArray[i];
    }
    int bTemp;
    for (int i = 0; i < iFilterLen - 1; i++)
    {
        for (int j = 0; j < iFilterLen - i - 1; j++)
        {
            if (bTab[j] > bTab[j + 1])
            {
                bTemp = bTab[j];
                bTab[j] = bTab[j + 1];
                bTab[j + 1] = bTemp;
            }
        }
    }
    if ((iFilterLen & 1) > 0)
    {
        bTemp = bTab[(iFilterLen - 1) / 2];
    }
    else
    {
        bTemp = (bTab[iFilterLen / 2] + bTab[iFilterLen / 2 - 1]) / 2;
    }
    return bTemp;
}

void Do(void *arg)
{
#define SCOUNT 20
    int analogBuffer[SCOUNT];
    int analogBufferTemp[SCOUNT];
    int analogBufferIndex = 0;
    int count = 0;
    int a;
    char temperature[10];
    r = R0 * exp(-B / T0);
    while (1)
    {
        analogBuffer[analogBufferIndex] = adc1_get_raw(ADC1_CHANNEL_4);
        analogBufferIndex++;
        vTaskDelay(50 / portTICK_RATE_MS);
        if (analogBufferIndex == SCOUNT)
        {
            analogBufferIndex = 0;
            for (int copyIndex = 0; copyIndex < SCOUNT; copyIndex++)
            {
                analogBufferTemp[copyIndex] = analogBuffer[copyIndex];
            }
            V1 = (float)getMedianNum(analogBufferTemp, SCOUNT);
            R2 = R1 * (V0 / V1 - 1);
            T = B / log(R2 / r) - 273.15;
            count++;
            if (count == SCOUNT / 2)
            {
                if (MQTT_CONNECTED)
                {
                    count = 0;
                    sprintf(temperature, "%.2f", T);
                    esp_mqtt_client_publish(client, "channels/2154705/publish/fields/field1", temperature, 0, 0, 0);
                    printf("publish thanh cong\n ");
                }
                else
                {
                    ESP_LOGE(TAG, "MQTT Not connected");
                }
            }
            if (T < 20)
            {
                a = 1;
                xQueueSend(queue, &a, (TickType_t)0);
                xQueueSend(queue, &a, (TickType_t)0); // ghi dữ liệu vào hàng đợi
            }
            else if (T >= 20 && T <= 30)
            {
                a = 2;
                xQueueSend(queue, &a, (TickType_t)0);
                xQueueSend(queue, &a, (TickType_t)0);
            }
            else
            {
                a = 3;
                xQueueSend(queue, &a, (TickType_t)0);
                xQueueSend(queue, &a, (TickType_t)0);
            }
        }
    }
}

void Nhay1(void *arg)
{
    int b; // tạo kí tự để lưu trữ dữ liệu sẽ nhận từ hàng đợi
    while (1)
    {
        xQueueReceive(queue, &b, (TickType_t)1000); // đọc dữ liệu từ hàng đợi
        if (b == 1)
        {
            printf("Received data from queue == %d, ", b);
            gpio_set_level(Den1, 1);
            vTaskDelay(500 / portTICK_RATE_MS);
            gpio_set_level(Den1, 0);
            vTaskDelay(500 / portTICK_RATE_MS);
        }
        else if (b == 2)
        {
            printf("Received data from queue == %d, ", b);
            gpio_set_level(Den1, 1);
            vTaskDelay(500 / portTICK_RATE_MS);
            gpio_set_level(Den1, 0);
            vTaskDelay(500 / portTICK_RATE_MS);
        }
        else if (b == 3)
        {
            printf("Received data from queue == %d, ", b);
            gpio_set_level(Den1, 0);
            vTaskDelay(1000 / portTICK_RATE_MS);
        }
    }
}

void Nhay2(void *arg)
{
    int c; // tạo kí tự để lưu trữ dữ liệu sẽ nhận từ hàng đợi
    while (1)
    {
        xQueueReceive(queue, &c, (TickType_t)1000); // đọc dữ liệu từ hàng đợi
        if (c == 1)
        {
            printf("T1= %f, V1=%f\n", T, V1);
            gpio_set_level(Den2, 0);
            vTaskDelay(500 / portTICK_RATE_MS);
        }
        else if (c == 2)
        {
            printf("T2= %f, V1=%f\n", T, V1);
            gpio_set_level(Den2, 1);
            vTaskDelay(250 / portTICK_RATE_MS);
            gpio_set_level(Den2, 0);
            vTaskDelay(250 / portTICK_RATE_MS);
            gpio_set_level(Den2, 1);
            vTaskDelay(250 / portTICK_RATE_MS);
            gpio_set_level(Den2, 0);
            vTaskDelay(250 / portTICK_RATE_MS);
        }
        else if (c == 3)
        {
            printf("T3= %f, V1=%f\n", T, V1);
            gpio_set_level(Den2, 1);
            vTaskDelay(125 / portTICK_RATE_MS);
            gpio_set_level(Den2, 0);
            vTaskDelay(125 / portTICK_RATE_MS);
            gpio_set_level(Den2, 1);
            vTaskDelay(125 / portTICK_RATE_MS);
            gpio_set_level(Den2, 0);
            vTaskDelay(125 / portTICK_RATE_MS);
            gpio_set_level(Den2, 1);
            vTaskDelay(125 / portTICK_RATE_MS);
            gpio_set_level(Den2, 0);
            vTaskDelay(125 / portTICK_RATE_MS);
            gpio_set_level(Den2, 1);
            vTaskDelay(125 / portTICK_RATE_MS);
            gpio_set_level(Den2, 0);
            vTaskDelay(125 / portTICK_RATE_MS);
        }
    }
}

void app_main(void)
{
    gpio_pad_select_gpio(Den1); // đặt cấu hình Den1 làm GPIO
    gpio_pad_select_gpio(Den2);
    gpio_set_direction(Den1, GPIO_MODE_OUTPUT); // đặt Den1 làm đầu ra
    gpio_set_direction(Den2, GPIO_MODE_OUTPUT);
    esp_adc_cal_characterize(ADC_UNIT_1, ADC_ATTEN_DB_11, ADC_WIDTH_BIT_DEFAULT, 0, &adc1_chars); // hiệu chỉnh ADC1 ở mức suy giảm 11db

    adc1_config_width(ADC_WIDTH_BIT_DEFAULT);                   // đặt cấu hình ADC1 ở độ rộng bit mặc định (12bit)
    adc1_config_channel_atten(ADC1_CHANNEL_4, ADC_ATTEN_DB_11); // đặt tham số suy giảm của ADC1 kênh 4 là GPIO32 thành 11db

    queue = xQueueCreate(5, sizeof(int)); // tạo hàng đợi

    xTaskCreate(Do, "Do", 2048, NULL, 10, NULL);
    xTaskCreatePinnedToCore(Nhay1, "Nhay1", 2048, NULL, 10, NULL, 0);
    xTaskCreatePinnedToCore(Nhay2, "Nhay2", 2048, NULL, 10, NULL, 0);
    ESP_ERROR_CHECK(i2c_master_init());
    ESP_LOGI(TAG, "I2C initialized successfully");
    esp_err_t ret = nvs_flash_init();
    if (ret == ESP_ERR_NVS_NO_FREE_PAGES || ret == ESP_ERR_NVS_NEW_VERSION_FOUND)
    {
        ESP_ERROR_CHECK(nvs_flash_erase());
        ret = nvs_flash_init();
    }
    ESP_ERROR_CHECK(ret);
    wifi_init();

    struct tm time = {
        .tm_year = 123,
        .tm_mon = 04,
        .tm_mday = 17,
        .tm_wday = 03,
        .tm_hour = 11,
        .tm_min = 30,
        .tm_sec = 30,
    };
    ds1307_set_time(&time);
    lcd_init();
    lcd_clear();
    while (1)
    {
        ds1307_get_time(&time);
        const char *day_of_week = get_day_of_week(&time);
        const char *time_now = get_time_string(&time);
        printf("%02d:%02d:%02d %s %02d/%02d/%04d \n", time.tm_hour, time.tm_min, time.tm_sec, day_of_week,
               time.tm_mday, time.tm_mon + 1, time.tm_year + 1900);
        sprintf(buffer1, "T=%.2f", T);
        lcd_put_cur(1, 0);
        lcd_send_string(buffer1);
        sprintf(buffer0, "%s", time_now);
        lcd_put_cur(0, 0);
        lcd_send_string(buffer0);
        vTaskDelay(1000 / portTICK_RATE_MS);
    }
}
````

````C
#include "driver/i2c.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>

#include "freertos/FreeRTOS.h"
#include "freertos/queue.h"
#include "freertos/task.h"
#include "driver/gpio.h"
#include "esp_adc_cal.h"
#include "esp_system.h"

#include "esp_http_client.h"
#include "connect_wifi.h"
#include "esp_log.h"
#include "nvs_flash.h"
#include "esp_event.h"
#include "esp_netif.h"
#include "../components/myTDS.h"

static const char *TAG = "HTTP_CLIENT";
char api_key[] = "EH18IFEEIV9JKOHH";

#define SCOUNT 30
float T, DObh, ntu, tds;

static esp_adc_cal_characteristics_t adc1_chars;
QueueHandle_t queue;

int getMedianNum(int bArray[], int iFilterLen)
{
  int bTab[iFilterLen];
  for (int i = 0; i < iFilterLen; i++)
  {
    bTab[i] = bArray[i];
  }
  int bTemp;
  for (int i = 0; i < iFilterLen - 1; i++)
  {
    for (int j = 0; j < iFilterLen - i - 1; j++)
    {
      if (bTab[j] > bTab[j + 1])
      {
        bTemp = bTab[j];
        bTab[j] = bTab[j + 1];
        bTab[j + 1] = bTemp;
      }
    }
  }
  if ((iFilterLen & 1) > 0)
  {
    bTemp = bTab[(iFilterLen - 1) / 2];
  }
  else
  {
    bTemp = (bTab[iFilterLen / 2] + bTab[iFilterLen / 2 - 1]) / 2;
  }
  return bTemp;
}

void DO_bh(void *arg)
{
  float V0 = 4095, R1 = 5000, R0 = 10000, B = 3950, T0 = 298.15;
  float V1 = 0, R2, r;
  int analogBuffer[SCOUNT];
  int analogBufferTemp[SCOUNT];
  int analogBufferIndex = 0;
  int a = 1;
  r = R0 * exp(-B / T0);
  while (1)
  {
    analogBuffer[analogBufferIndex] = adc1_get_raw(ADC1_CHANNEL_4);
    analogBufferIndex++;
    vTaskDelay(40 / portTICK_RATE_MS);
    if (analogBufferIndex == SCOUNT)
    {
      analogBufferIndex = 0;
      for (int copyIndex = 0; copyIndex < SCOUNT; copyIndex++)
      {
        analogBufferTemp[copyIndex] = analogBuffer[copyIndex];
      }
      V1 = (float)getMedianNum(analogBufferTemp, SCOUNT);
      R2 = R1 * (V0 / V1 - 1);
      T = B / log(R2 / r) - 273.15;
      xQueueSend(queue, &a, (TickType_t)0);
      DObh = 14.652 - 0.41022 * T + 0.007991 * pow(T, 2) - 0.000077774 * pow(T, 3);
      printf("T=%f, DObh=%f\n ", T, DObh);
      vTaskDelay(1000 / portTICK_RATE_MS);
    }
  }
}

void NTU(void *arg)
{
  float V0 = 5;
  float V1, V2;
  int analogBuffer[SCOUNT];
  int analogBufferTemp[SCOUNT];
  int analogBufferIndex = 0;
  while (1)
  {
    analogBuffer[analogBufferIndex] = adc1_get_raw(ADC1_CHANNEL_5);
    analogBufferIndex++;
    vTaskDelay(40 / portTICK_RATE_MS);
    if (analogBufferIndex == SCOUNT)
    {
      analogBufferIndex = 0;
      for (int copyIndex = 0; copyIndex < SCOUNT; copyIndex++)
      {
        analogBufferTemp[copyIndex] = analogBuffer[copyIndex];
      }
      V1 = (float)getMedianNum(analogBufferTemp, SCOUNT) * V0 / 4095;
      V2 = V0 - V1;
      if (V2 < 2.5)
      {
        ntu = 3000;
      }
      else
      {
        ntu = -1120.4 * pow(V2, 2) + 5742.3 * V2 - 4353.8;
      }
      printf("ntu=%f\n", ntu);
      vTaskDelay(1000 / portTICK_RATE_MS);
    }
  }
}

void TDS(void *arg)
{
  float V0 = 5;
  float V3;
  int b;
  int analogBuffer[SCOUNT];
  int analogBufferTemp[SCOUNT];
  int analogBufferIndex = 0;
  xQueueReceive(queue, &b, (TickType_t)1000);
  while (b == 1)
  {
    analogBuffer[analogBufferIndex] = adc1_get_raw(ADC1_CHANNEL_6);
    analogBufferIndex++;
    vTaskDelay(40 / portTICK_RATE_MS);
    if (analogBufferIndex == SCOUNT)
    {
      analogBufferIndex = 0;
      for (int copyIndex = 0; copyIndex < SCOUNT; copyIndex++)
      {
        analogBufferTemp[copyIndex] = analogBuffer[copyIndex];
      }
      V3 = (float)getMedianNum(analogBufferTemp, SCOUNT) * V0 / 4095;
      float compensationCoefficient = 1.0 + 0.02 * (T - 25.0);
      float compensationVolatge = V3 / compensationCoefficient;
      tds = (133.42 * pow(compensationVolatge, 3) - 255.86 * pow(compensationVolatge, 2) + 857.39 * compensationVolatge) * 0.5;
      printf("tds=%f\n ", tds);
      vTaskDelay(1000 / portTICK_RATE_MS);
    }
  }
}

void send_data_to_thingspeak(void *pvParameters)
{
  char thingspeak_url[] = "http://192.168.1.160:3000";
  char data[] = "/upload?Nhietdo=%.2f&DO_baohoa=%.2f&NTU=%.2f&WQI_NTU=%.2f&TDS=%.2f&WQI_TDS=%.2f";
  char post_data[200];
  esp_err_t err;

  esp_http_client_config_t config = {
      .url = thingspeak_url,
      .method = HTTP_METHOD_GET,
  };
  esp_http_client_handle_t client = esp_http_client_init(&config);
  esp_http_client_set_header(client, "Content-Type", "application/x-www-form-urlencoded");
  while (1)
  {
    vTaskDelay(2000 / portTICK_RATE_MS);
    strcpy(post_data, "");
    snprintf(post_data, sizeof(post_data), data, T, DObh, ntu, 0, tds, 0);
    ESP_LOGI(TAG, "post = %s", post_data);
    esp_http_client_set_post_field(client, post_data, strlen(post_data));
    esp_http_client_set_url(client, post_data);
    err = esp_http_client_perform(client);

    if (err == ESP_OK)
    {
      int status_code = esp_http_client_get_status_code(client);
      if (status_code == 200)
      {
        ESP_LOGI(TAG, "Message sent Successfully");
      }
      else
      {
        ESP_LOGI(TAG, "Message sent Failed");
        goto exit;
      }
    }
    else
    {
      ESP_LOGI(TAG, "Message sent Failed");
      goto exit;
    }
  }
exit:
  esp_http_client_cleanup(client);
  vTaskDelete(NULL);
}
void app_main(void)
{
  esp_adc_cal_characterize(ADC_UNIT_1, ADC_ATTEN_DB_11, ADC_WIDTH_BIT_DEFAULT, 0, &adc1_chars); // hiệu chỉnh ADC1 ở mức suy giảm 11db
  adc1_config_width(ADC_WIDTH_BIT_DEFAULT);                                                     // đặt cấu hình ADC1 ở độ rộng bit mặc định (12bit)
  adc1_config_channel_atten(ADC1_CHANNEL_4, ADC_ATTEN_DB_11);                                   // đặt tham số suy giảm của ADC1 kênh 4 là GPIO32 thành 11db
  adc1_config_channel_atten(ADC1_CHANNEL_5, ADC_ATTEN_DB_11);
  adc1_config_channel_atten(ADC1_CHANNEL_6, ADC_ATTEN_DB_11);
  queue = xQueueCreate(5, sizeof(int));

  esp_err_t ret = nvs_flash_init();
  if (ret == ESP_ERR_NVS_NO_FREE_PAGES || ret == ESP_ERR_NVS_NEW_VERSION_FOUND)
  {
    ESP_ERROR_CHECK(nvs_flash_erase());
    ret = nvs_flash_init();
  }
  ESP_ERROR_CHECK(ret);

  connect_wifi();
  if (wifi_connect_status)
  {
    xTaskCreatePinnedToCore(DO_bh, "DO_bh", 2048, NULL, 10, NULL, 0);
    xTaskCreatePinnedToCore(NTU, "NTU", 2048, NULL, 10, NULL, 1);
    xTaskCreatePinnedToCore(TDS, "TDS", 2048, NULL, 10, NULL, 0);

    xTaskCreate(&send_data_to_thingspeak, "send_data_to_thingspeak", 8192, NULL, 10, NULL);
  }
}

````C
#include "driver/i2c.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>

#include "freertos/FreeRTOS.h"
#include "freertos/queue.h"
#include "freertos/task.h"
#include "driver/gpio.h"
#include "esp_adc_cal.h"
#include "esp_system.h"

#include "esp_http_client.h"
#include "../components/connect_wifi.h"
#include "esp_log.h"
#include "nvs_flash.h"
#include "esp_event.h"
#include "esp_netif.h"

static const char *TAG = "HTTP_CLIENT";
char api_key[] = "EH18IFEEIV9JKOHH";

#define SCOUNT 30
float T, DObh, ntu, WQIdoduc, tds;

static esp_adc_cal_characteristics_t adc1_chars;
QueueHandle_t queue;

int getMedianNum(int bArray[], int iFilterLen)
{
  int bTab[iFilterLen];
  for (int i = 0; i < iFilterLen; i++)
  {
    bTab[i] = bArray[i];
  }
  int bTemp;
  for (int i = 0; i < iFilterLen - 1; i++)
  {
    for (int j = 0; j < iFilterLen - i - 1; j++)
    {
      if (bTab[j] > bTab[j + 1])
      {
        bTemp = bTab[j];
        bTab[j] = bTab[j + 1];
        bTab[j + 1] = bTemp;
      }
    }
  }
  if ((iFilterLen & 1) > 0)
  {
    bTemp = bTab[(iFilterLen - 1) / 2];
  }
  else
  {
    bTemp = (bTab[iFilterLen / 2] + bTab[iFilterLen / 2 - 1]) / 2;
  }
  return bTemp;
}

float WQI_doduc(float Cp){
  float WQI_doduc;
  if(Cp<=5){
    WQI_doduc=100;
  }
  if(5<Cp&&Cp<20){
    WQI_doduc=(100-75)/(20-5)*(20-Cp)+75;
  }
  if(Cp==20){
    WQI_doduc=75;
  }
  if(20<Cp&&Cp<30){
    WQI_doduc=(75-50)/(30-20)*(30-Cp)+50;
  }
  if(Cp==30){
    WQI_doduc=50;
  }
  if(30<Cp&&Cp<70){
    WQI_doduc=(50-25)/(70-30)*(70-Cp)+25;
  }
  if(Cp==70){
    WQI_doduc=25;
  }
  if(70<Cp&&Cp<100){
    WQI_doduc=(25-1)/(100-70)*(100-Cp)+1;
  }
  if(Cp>=100){
    WQI_doduc=1;
  }
  return WQI_doduc;
}
void DO_bh(void *arg)
{
  float V0 = 4095, R1 = 5000, R0 = 10000, B = 3950, T0 = 298.15;
  float V1 = 0, R2, r;
  int analogBuffer[SCOUNT];
  int analogBufferTemp[SCOUNT];
  int analogBufferIndex = 0;
  int a = 1;
  r = R0 * exp(-B / T0);
  while (1)
  {
    analogBuffer[analogBufferIndex] = adc1_get_raw(ADC1_CHANNEL_4);
    analogBufferIndex++;
    vTaskDelay(40 / portTICK_RATE_MS);
    if (analogBufferIndex == SCOUNT)
    {
      analogBufferIndex = 0;
      for (int copyIndex = 0; copyIndex < SCOUNT; copyIndex++)
      {
        analogBufferTemp[copyIndex] = analogBuffer[copyIndex];
      }
      V1 = (float)getMedianNum(analogBufferTemp, SCOUNT);
      R2 = R1 * (V0 / V1 - 1);
      T = B / log(R2 / r) - 273.15;
      xQueueSend(queue, &a, (TickType_t)0);
      DObh = 14.652 - 0.41022 * T + 0.007991 * pow(T, 2) - 0.000077774 * pow(T, 3);
      printf("T=%f, DObh=%f\n ", T, DObh);
      vTaskDelay(1000 / portTICK_RATE_MS);
    }
  }
}

void NTU(void *arg)
{
  float V0 = 5;
  float V1, V2;
  int analogBuffer[SCOUNT];
  int analogBufferTemp[SCOUNT];
  int analogBufferIndex = 0;
  while (1)
  {
    analogBuffer[analogBufferIndex] = adc1_get_raw(ADC1_CHANNEL_5);
    analogBufferIndex++;
    vTaskDelay(40 / portTICK_RATE_MS);
    if (analogBufferIndex == SCOUNT)
    {
      analogBufferIndex = 0;
      for (int copyIndex = 0; copyIndex < SCOUNT; copyIndex++)
      {
        analogBufferTemp[copyIndex] = analogBuffer[copyIndex];
      }
      V1 = (float)getMedianNum(analogBufferTemp, SCOUNT) * V0 / 4095;
      V2 = V0 - V1;
      if(V2 < 2.5){
        ntu = 3000;
      }
      else{
        ntu = -1120.4*pow(V2,2)+5742.3*V2-4352.9; 
      }
      WQIdoduc= WQI_doduc(ntu);
      printf("ntu=%f\n", ntu);
      printf("WQIdoduc=%f\n", WQIdoduc);
      vTaskDelay(1000 / portTICK_RATE_MS);
    }
  }
}

void TDS(void *arg)
{
  float V0 = 5;
  float V3;
  int b;
  int analogBuffer[SCOUNT];
  int analogBufferTemp[SCOUNT];
  int analogBufferIndex = 0;
  xQueueReceive(queue, &b, (TickType_t)1000);
  while (b == 1)
  {
    analogBuffer[analogBufferIndex] = adc1_get_raw(ADC1_CHANNEL_6);
    analogBufferIndex++;
    vTaskDelay(40 / portTICK_RATE_MS);
    if (analogBufferIndex == SCOUNT)
    {
      analogBufferIndex = 0;
      for (int copyIndex = 0; copyIndex < SCOUNT; copyIndex++)
      {
        analogBufferTemp[copyIndex] = analogBuffer[copyIndex];
      }
      V3 = (float)getMedianNum(analogBufferTemp, SCOUNT) * V0 / 4095;
      float compensationCoefficient = 1.0 + 0.02 * (T - 25.0);
      float compensationVolatge = V3 / compensationCoefficient;
      tds = (133.42 * pow(compensationVolatge, 3) - 255.86 * pow(compensationVolatge, 2) + 857.39 * compensationVolatge) * 0.5;
      printf("tds=%f\n ", tds);
      vTaskDelay(1000 / portTICK_RATE_MS);
    }
  }
}

void send_data_to_thingspeak(void *pvParameters)
{
  char thingspeak_url[] = "http://192.168.1.8:3000";
  char data[] = "/upload?Nhietdo=%.2f&DO_baohoa=%.2f&NTU=%.2f&WQI_NTU=%.2f&TDS=%.2f";
  char post_data[200];
  esp_err_t err;

  esp_http_client_config_t config = {
      .url = thingspeak_url,
      .method = HTTP_METHOD_GET,
  };
  esp_http_client_handle_t client = esp_http_client_init(&config);
  esp_http_client_set_header(client, "Content-Type", "application/x-www-form-urlencoded");
  while (1)
  {
    vTaskDelay(2000 / portTICK_RATE_MS);
    strcpy(post_data, "");
    snprintf(post_data, sizeof(post_data), data, T, DObh, ntu, WQIdoduc, tds);
    ESP_LOGI(TAG, "post = %s", post_data);
    esp_http_client_set_post_field(client, post_data, strlen(post_data));
    esp_http_client_set_url(client, post_data);
    err = esp_http_client_perform(client);

    if (err == ESP_OK)
    {
      int status_code = esp_http_client_get_status_code(client);
      if (status_code == 200)
      {
        ESP_LOGI(TAG, "Message sent Successfully");
      }
      else
      {
        ESP_LOGI(TAG, "Message sent Failed");
        goto exit;
      }
    }
    else
    {
      ESP_LOGI(TAG, "Message sent Failed");
      goto exit;
    }
  }
exit:
  esp_http_client_cleanup(client);
  vTaskDelete(NULL);
}
void app_main(void)
{
  esp_adc_cal_characterize(ADC_UNIT_1, ADC_ATTEN_DB_11, ADC_WIDTH_BIT_DEFAULT, 0, &adc1_chars); // hiệu chỉnh ADC1 ở mức suy giảm 11db
  adc1_config_width(ADC_WIDTH_BIT_DEFAULT);                                                     // đặt cấu hình ADC1 ở độ rộng bit mặc định (12bit)
  adc1_config_channel_atten(ADC1_CHANNEL_4, ADC_ATTEN_DB_11);                                   // đặt tham số suy giảm của ADC1 kênh 4 là GPIO32 thành 11db
  adc1_config_channel_atten(ADC1_CHANNEL_5, ADC_ATTEN_DB_11);
  adc1_config_channel_atten(ADC1_CHANNEL_6, ADC_ATTEN_DB_11);
  queue = xQueueCreate(5, sizeof(int));

  esp_err_t ret = nvs_flash_init();
  if (ret == ESP_ERR_NVS_NO_FREE_PAGES || ret == ESP_ERR_NVS_NEW_VERSION_FOUND)
  {
    ESP_ERROR_CHECK(nvs_flash_erase());
    ret = nvs_flash_init();
  }
  ESP_ERROR_CHECK(ret);

  connect_wifi();
  if (wifi_connect_status)
  {
    xTaskCreatePinnedToCore(DO_bh, "DO_bh", 2048, NULL, 10, NULL, 0);
    xTaskCreatePinnedToCore(NTU, "NTU", 2048, NULL, 10, NULL, 1);
    xTaskCreatePinnedToCore(TDS, "TDS", 2048, NULL, 10, NULL, 0);

    xTaskCreate(&send_data_to_thingspeak, "send_data_to_thingspeak", 8192, NULL, 10, NULL);
  }
}

````C
#include <stdio.h>
#include <stdbool.h>
#include <unistd.h>
#include <math.h>
#include "../components/myTDS.h"
nvs_handle_t nvsHandle;
void app_main(void)
{ 
  float TDS_val; 
  nvs_flash_init();
	//TDS_calib(nvsHandle, ADC1_CHANNEL_6, 5000.0, 4096.0, "storage", 111.0);
  TDS_init_param(&nvsHandle);
  while(1){
    TDS_val = get_TDS_toSalt(ADC1_CHANNEL_6, 25.0, 0.5, 4095.0, 5000.0);
	  printf("\nTDS value:%f\n",TDS_val);
    vTaskDelay(1000/portTICK_PERIOD_MS);
  }
}


````C
#include "driver/i2c.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>

#include "freertos/FreeRTOS.h"
#include "freertos/queue.h"
#include "freertos/task.h"
#include "driver/gpio.h"
#include "esp_adc_cal.h"
#include "esp_system.h"

#include "esp_http_client.h"
#include "../components/connect_wifi.h"
#include "esp_log.h"
#include "nvs_flash.h"
#include "esp_event.h"
#include "esp_netif.h"

static const char *TAG = "HTTP_CLIENT";
#define SCOUNT 50
float T, DObh, ntu, WQIdoduc, tds;

static esp_adc_cal_characteristics_t adc1_chars;
QueueHandle_t queue;

int getMedianNum(int bArray[], int iFilterLen, adc1_channel_t adc_chan)
{
  int adc=0;
  for (int i = 0; i < iFilterLen; i++)
  {
    bArray[i]=adc1_get_raw(adc_chan);
    vTaskDelay(100 / portTICK_RATE_MS);
  }
  for (int i = 0; i < iFilterLen - 1; i++)
  {
    for (int j = 0; j < iFilterLen - i - 1; j++)
    {
      if (bArray[j] > bArray[j + 1])
      {
        int bTemp = bArray[j];
        bArray[j] = bArray[j + 1];
        bArray[j + 1] = bTemp;
      }
    }
  }
  for(int i = 0; i <iFilterLen ; i ++)
  {
		adc += bArray[i];
	}
	adc /=iFilterLen;
  return adc;
}

float WQI_doduc(float Cp){
  float WQI_doduc;
  if(Cp<=5){
    WQI_doduc=100;
  }
  if(5<Cp&&Cp<20){
    WQI_doduc=(float)(100-75)*(20-Cp)/(20-5)+75;
  }
  if(Cp==20){
    WQI_doduc=75;
  }
  if(20<Cp&&Cp<30){
    WQI_doduc=(float)(75-50)*(30-Cp)/(30-20)+50;
  }
  if(Cp==30){
    WQI_doduc=50;
  }
  if(30<Cp&&Cp<70){
    WQI_doduc=(float)(50-25)*(70-Cp)/(70-30)+25;
  }
  if(Cp==70){
    WQI_doduc=25;
  }
  if(70<Cp&&Cp<100){
    WQI_doduc=(float)(25-1)*(100-Cp)/(100-70)+1;
  }
  if(Cp>=100){
    WQI_doduc=1;
  }
  return WQI_doduc;
}
void DO_bh(void *arg)
{
  float V0 = 4095, R1 = 5000, R0 = 10000, B = 3950, T0 = 298.15;
  float V1 = 0, R2, r;
  int analogBuffer[SCOUNT];
  int a = 1;
  r = R0 * exp(-B / T0);
  while (1)
  {
      V1 = getMedianNum(analogBuffer, SCOUNT, ADC1_CHANNEL_4);
      R2 = R1 * (V0 / V1 - 1);
      T = B / log(R2 / r) - 273.15;
      DObh = 14.652 - 0.41022 * T + 0.007991 * pow(T, 2) - 0.000077774 * pow(T, 3);
      printf("T=%f, DObh=%f\n ", T, DObh);
      xQueueSend(queue, &a, (TickType_t)0);
      //vTaskDelay(1000 / portTICK_RATE_MS);
  }
}

void NTU(void *arg)
{
  float V1, V2;
  int analogBuffer[SCOUNT];
  while (1)
  {
      V1 = (float)getMedianNum(analogBuffer, SCOUNT, ADC1_CHANNEL_5) * 5 / 4095;
      if(V1 < 2.5){
        ntu = 3000;
      }
      else{
        ntu = -1120.4*pow(V1,2)+5742.3*V1-4352.9; 
      }
      WQIdoduc= WQI_doduc(ntu);
      printf("V1=%f, V2=%f, ntu=%f, WQIdoduc=%f\n", V1, V2, ntu, WQIdoduc);
      //vTaskDelay(1000 / portTICK_RATE_MS);
  }
}

void TDS(void *arg)
{
  float V3;
  int analogBuffer[SCOUNT];
  while (1)
  {
      V3 = (float)getMedianNum(analogBuffer, SCOUNT, ADC1_CHANNEL_6) * 5 / 4095;
      float compensationCoefficient = 1.0 + 0.02 * (T - 25.0);
      float compensationVolatge = V3 / compensationCoefficient;
      tds = (133.42 * pow(compensationVolatge, 3) - 255.86 * pow(compensationVolatge, 2) + 857.39 * compensationVolatge) * 0.5;
      printf("tds=%f\n ", tds);
      //printf("V3=%f\n ", V3);
      //vTaskDelay(1000 / portTICK_RATE_MS);
    
  }
}

void send_data_to_pwm(void *pvParameters)
{
  char pwm_url[] = "http://192.168.1.160:3000";
  char data[] = "/upload?Nhietdo=%.2f&DO_baohoa=%.2f&NTU=%.2f&WQI_NTU=%.2f&TDS=%.2f";
  char post_data[200];
  esp_err_t err;

  esp_http_client_config_t config = {
      .url = pwm_url,
      .method = HTTP_METHOD_GET,
  };
  esp_http_client_handle_t client = esp_http_client_init(&config);
  esp_http_client_set_header(client, "Content-Type", "application/x-www-form-urlencoded");
  while (1)
  {
    vTaskDelay(4000 / portTICK_RATE_MS);
    strcpy(post_data, "");
    snprintf(post_data, sizeof(post_data), data, T, DObh, ntu, WQIdoduc, tds);
    ESP_LOGI(TAG, "post = %s", post_data);
    esp_http_client_set_post_field(client, post_data, strlen(post_data));
    esp_http_client_set_url(client, post_data);
    err = esp_http_client_perform(client);

    if (err == ESP_OK)
    {
      int status_code = esp_http_client_get_status_code(client);
      if (status_code == 200)
      {
        ESP_LOGI(TAG, "Message sent Successfully");
      }
      else
      {
        ESP_LOGI(TAG, "Message sent Failed");
        goto exit;
      }
    }
    else
    {
      ESP_LOGI(TAG, "Message sent Failed");
      goto exit;
    }
  }
exit:
  esp_http_client_cleanup(client);
  vTaskDelete(NULL);
}
void app_main(void)
{
  esp_adc_cal_characterize(ADC_UNIT_1, ADC_ATTEN_DB_11, ADC_WIDTH_BIT_DEFAULT, 0, &adc1_chars); // hiệu chỉnh ADC1 ở mức suy giảm 11db
  adc1_config_width(ADC_WIDTH_BIT_DEFAULT);                                                     // đặt cấu hình ADC1 ở độ rộng bit mặc định (12bit)
  adc1_config_channel_atten(ADC1_CHANNEL_4, ADC_ATTEN_DB_11);                                   // đặt tham số suy giảm của ADC1 kênh 4 là GPIO32 thành 11db
  adc1_config_channel_atten(ADC1_CHANNEL_5, ADC_ATTEN_DB_11);
  adc1_config_channel_atten(ADC1_CHANNEL_6, ADC_ATTEN_DB_11);
  queue = xQueueCreate(5, sizeof(int));

  esp_err_t ret = nvs_flash_init();
  if (ret == ESP_ERR_NVS_NO_FREE_PAGES || ret == ESP_ERR_NVS_NEW_VERSION_FOUND)
  {
    ESP_ERROR_CHECK(nvs_flash_erase());
    ret = nvs_flash_init();
  }
  ESP_ERROR_CHECK(ret);

  connect_wifi();
  if (wifi_connect_status)
  {
    xTaskCreatePinnedToCore(DO_bh, "DO_bh", 2048, NULL, 10, NULL, 0);
    xTaskCreatePinnedToCore(NTU, "NTU", 2048, NULL, 10, NULL, 1);
    xTaskCreatePinnedToCore(TDS, "TDS", 2048, NULL, 10, NULL, 0);

    xTaskCreate(&send_data_to_pwm, "send_data_to_pwm", 8192, NULL, 10, NULL);
  }
}


````C
#include "driver/i2c.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>

#include "freertos/FreeRTOS.h"
#include "freertos/queue.h"
#include "freertos/task.h"
#include "driver/gpio.h"
#include "esp_adc_cal.h"
#include "esp_system.h"

#include "esp_http_client.h"
#include "../components/connect_wifi.h"
#include "../components/myTDS.h"
#include "../components/nvs_interface.h"
#include "esp_log.h"
#include "nvs_flash.h"
#include "esp_event.h"
#include "esp_netif.h"

static const char *TAG = "HTTP_CLIENT";
#define SCOUNT 50
float T, DObh, ntu, WQIdoduc, tds;
float V0 = 4095, R1 = 5000, R0 = 10000, B = 3950, T0 = 298.15;
float V1, R2, r;
int analogBuffer[SCOUNT];

static esp_adc_cal_characteristics_t adc1_chars;
QueueHandle_t queue;
nvs_handle_t nvsHandle;

int getMedianNum(int bArray[], int iFilterLen, adc1_channel_t adc_chan)
{
  int adc=0;
  for (int i = 0; i < iFilterLen; i++)
  {
    bArray[i]=adc1_get_raw(adc_chan);
    vTaskDelay(100 / portTICK_RATE_MS);
  }
  for (int i = 0; i < iFilterLen - 1; i++)
  {
    for (int j = 0; j < iFilterLen - i - 1; j++)
    {
      if (bArray[j] > bArray[j + 1])
      {
        int bTemp = bArray[j];
        bArray[j] = bArray[j + 1];
        bArray[j + 1] = bTemp;
      }
    }
  }
  for(int i = 0; i <iFilterLen ; i ++)
  {
		adc += bArray[i];
	}
	adc /=iFilterLen;
  return adc;
}

float WQI_doduc(float Cp){
  float WQI_doduc;
  if(Cp<=5){
    WQI_doduc=100;
  }
  if(5<Cp&&Cp<20){
    WQI_doduc=(float)(100-75)*(20-Cp)/(20-5)+75;
  }
  if(Cp==20){
    WQI_doduc=75;
  }
  if(20<Cp&&Cp<30){
    WQI_doduc=(float)(75-50)*(30-Cp)/(30-20)+50;
  }
  if(Cp==30){
    WQI_doduc=50;
  }
  if(30<Cp&&Cp<70){
    WQI_doduc=(float)(50-25)*(70-Cp)/(70-30)+25;
  }
  if(Cp==70){
    WQI_doduc=25;
  }
  if(70<Cp&&Cp<100){
    WQI_doduc=(float)(25-1)*(100-Cp)/(100-70)+1;
  }
  if(Cp>=100){
    WQI_doduc=1;
  }
  return WQI_doduc;
}
void DO_bh(void *arg)
{
  int a = 1;
  int analogBuffer1[SCOUNT];
  r = R0 * exp(-B / T0);
  while (1)
  {
      V1 = getMedianNum(analogBuffer1, SCOUNT, ADC1_CHANNEL_4);
      R2 = R1 * (V0 / V1 - 1);
      T = B / log(R2 / r) - 273.15;
      DObh = 14.652 - 0.41022 * T + 0.007991 * pow(T, 2) - 0.000077774 * pow(T, 3);
      printf("T=%.2f, DObh=%.2f\n ", T, DObh);
      xQueueSend(queue, &a, (TickType_t)0);
      //vTaskDelay(1000 / portTICK_RATE_MS);
  }
}

void NTU(void *arg)
{
  float V2;
  int analogBuffer2[SCOUNT];
  while (1)
  {
      V2 = (float)getMedianNum(analogBuffer2, SCOUNT, ADC1_CHANNEL_5) * 5 / 4095;
      if(V2 < 2.5){
        ntu = 3000;
      }
      else{
        ntu = -1120.4*pow(V2,2)+5742.3*V2-4352.9; 
      }
      WQIdoduc= WQI_doduc(ntu);
      printf("V2=%.2f, ntu=%.2f, WQIdoduc=%.2f\n", V2, ntu, WQIdoduc);
      //vTaskDelay(1000 / portTICK_RATE_MS);
  }
}

void TDS(void *arg)
{
  while (1)
  {
    tds = get_TDS_toSalt(ADC1_CHANNEL_6, T, 0.5, 4095.0, 5000.0);
	  printf("\ntds=%.2f\n",tds);
  }
}

void send_data_to_pwm(void *pvParameters)
{
  int b;
  char pwm_url[] = "http://192.168.1.160:3000";
  char data[] = "/upload?Nhietdo=%.2f&DO_baohoa=%.2f&NTU=%.2f&WQI_NTU=%.2f&TDS=%.2f";
  char post_data[200];
  esp_err_t err;

  esp_http_client_config_t config = {
      .url = pwm_url,
      .method = HTTP_METHOD_GET,
  };
  esp_http_client_handle_t client = esp_http_client_init(&config);
  esp_http_client_set_header(client, "Content-Type", "application/x-www-form-urlencoded");
  while (1)
  {
    xQueueReceive(queue, &b, (TickType_t)1000);
    if(b==1)
    {
      vTaskDelay(500/ portTICK_RATE_MS);
      strcpy(post_data, "");
      snprintf(post_data, sizeof(post_data), data, T, DObh, ntu, WQIdoduc, tds);
      ESP_LOGI(TAG, "post = %s", post_data);
      esp_http_client_set_post_field(client, post_data, strlen(post_data));
      esp_http_client_set_url(client, post_data);
      err = esp_http_client_perform(client);

      if (err == ESP_OK)
      {
        int status_code = esp_http_client_get_status_code(client);
        if (status_code == 200)
        {
          ESP_LOGI(TAG, "Message sent Successfully");
        }
        else
        {
          ESP_LOGI(TAG, "Message sent Failed");
          goto exit;
        }
      }
      else
      {
        ESP_LOGI(TAG, "Message sent Failed1");
        goto exit;
      }
    }
  }
  exit:
    esp_http_client_cleanup(client);
    vTaskDelete(NULL);
}
void app_main(void)
{
  esp_adc_cal_characterize(ADC_UNIT_1, ADC_ATTEN_DB_11, ADC_WIDTH_BIT_DEFAULT, 0, &adc1_chars); // hiệu chỉnh ADC1 ở mức suy giảm 11db
  adc1_config_width(ADC_WIDTH_BIT_DEFAULT);                                                     // đặt cấu hình ADC1 ở độ rộng bit mặc định (12bit)
  adc1_config_channel_atten(ADC1_CHANNEL_4, ADC_ATTEN_DB_11);                                   // đặt tham số suy giảm của ADC1 kênh 4 là GPIO32 thành 11db
  adc1_config_channel_atten(ADC1_CHANNEL_5, ADC_ATTEN_DB_11);
  adc1_config_channel_atten(ADC1_CHANNEL_6, ADC_ATTEN_DB_11);
  queue = xQueueCreate(5, sizeof(int));

  esp_err_t ret = nvs_flash_init();
  if (ret == ESP_ERR_NVS_NO_FREE_PAGES || ret == ESP_ERR_NVS_NEW_VERSION_FOUND)
  {
    ESP_ERROR_CHECK(nvs_flash_erase());
    ret = nvs_flash_init();
  }
  ESP_ERROR_CHECK(ret);
  connect_wifi();
  // r = R0 * exp(-B / T0);
  // V1 = getMedianNum(analogBuffer, SCOUNT, ADC1_CHANNEL_4);
  // R2 = R1 * (V0 / V1 - 1);
  // T = B / log(R2 / r) - 273.15;
  // printf("T=%f\n ", T);
  //TDS_calib(nvsHandle, ADC1_CHANNEL_6, 5000.0, 4096.0, "storage", 111.0, T);
  TDS_init_param(&nvsHandle);
  if (wifi_connect_status)
  {
    xTaskCreatePinnedToCore(DO_bh, "DO_bh", 2048, NULL, 10, NULL, 0);
    xTaskCreatePinnedToCore(NTU, "NTU", 2048, NULL, 10, NULL, 1);
    xTaskCreatePinnedToCore(TDS, "TDS", 2048, NULL, 10, NULL, 0);

    xTaskCreate(&send_data_to_pwm, "send_data_to_pwm", 8192, NULL, 10, NULL);
  }
}


````C
#include "driver/i2c.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>

#include "freertos/FreeRTOS.h"
#include "freertos/queue.h"
#include "freertos/task.h"
#include "driver/gpio.h"
#include "esp_adc_cal.h"
#include "esp_system.h"

#include "esp_http_client.h"
#include "../components/connect_wifi.h"
#include "../components/myTDS.h"
#include "../components/nvs_interface.h"
#include "esp_log.h"
#include "nvs_flash.h"
#include "esp_event.h"
#include "esp_netif.h"

static const char *TAG = "HTTP_CLIENT";
#define SCOUNT 50
float T, DObh, ntu, WQIdoduc, tds;
float V0 = 4095, R1 = 5000, R0 = 10000, B = 3950, T0 = 298.15;
float V1, R2, r;
int analogBuffer[SCOUNT];

static esp_adc_cal_characteristics_t adc1_chars;
QueueHandle_t queue;
nvs_handle_t nvsHandle;

int getMedianNum(int bArray[], int iFilterLen, adc1_channel_t adc_chan)
{
  int adc=0;
  for (int i = 0; i < iFilterLen; i++)
  {
    bArray[i]=adc1_get_raw(adc_chan);
    vTaskDelay(100 / portTICK_RATE_MS);
  }
  for (int i = 0; i < iFilterLen - 1; i++)
  {
    for (int j = 0; j < iFilterLen - i - 1; j++)
    {
      if (bArray[j] > bArray[j + 1])
      {
        int bTemp = bArray[j];
        bArray[j] = bArray[j + 1];
        bArray[j + 1] = bTemp;
      }
    }
  }
  for(int i = 0; i <iFilterLen ; i ++)
  {
		adc += bArray[i];
	}
	adc /=iFilterLen;
  return adc;
}

float WQI_doduc(float Cp){
  float WQI_doduc;
  if(Cp<=5){
    WQI_doduc=100;
  }
  if(5<Cp&&Cp<20){
    WQI_doduc=(float)(100-75)*(20-Cp)/(20-5)+75;
  }
  if(Cp==20){
    WQI_doduc=75;
  }
  if(20<Cp&&Cp<30){
    WQI_doduc=(float)(75-50)*(30-Cp)/(30-20)+50;
  }
  if(Cp==30){
    WQI_doduc=50;
  }
  if(30<Cp&&Cp<70){
    WQI_doduc=(float)(50-25)*(70-Cp)/(70-30)+25;
  }
  if(Cp==70){
    WQI_doduc=25;
  }
  if(70<Cp&&Cp<100){
    WQI_doduc=(float)(25-1)*(100-Cp)/(100-70)+1;
  }
  if(Cp>=100){
    WQI_doduc=1;
  }
  return WQI_doduc;
}
void DO_bh(void *arg)
{
  int a = 1;
  int analogBuffer1[SCOUNT];
  r = R0 * exp(-B / T0);
  while (1)
  {
      V1 = getMedianNum(analogBuffer1, SCOUNT, ADC1_CHANNEL_4);
      R2 = R1 * (V0 / V1 - 1);
      T = B / log(R2 / r) - 273.15;
      DObh = 14.652 - 0.41022 * T + 0.007991 * pow(T, 2) - 0.000077774 * pow(T, 3);
      printf("T=%.2f, DObh=%.2f\n ", T, DObh);
      xQueueSend(queue, &a, (TickType_t)0);
      //vTaskDelay(1000 / portTICK_RATE_MS);
  }
}

void NTU(void *arg)
{
  float V2;
  int analogBuffer2[SCOUNT];
  while (1)
  {
      V2 = (float)getMedianNum(analogBuffer2, SCOUNT, ADC1_CHANNEL_5) * 5 / 4095;
      if(V2 < 2.5){
        ntu = 3000;
      }
      else{
        ntu = -1120.4*pow(V2,2)+5742.3*V2-4352.9; 
      }
      WQIdoduc= WQI_doduc(ntu);
      printf("ntu=%.2f, WQIdoduc=%.2f\n", ntu, WQIdoduc);
      //vTaskDelay(1000 / portTICK_RATE_MS);
  }
}

void TDS(void *arg)
{
  while (1)
  {
    tds = get_TDS_toSalt(ADC1_CHANNEL_6, T, 0.5, 4095.0, 5000.0);
	  printf("\ntds=%.2f\n",tds);
  }
}

void send_data_to_pwm(void *pvParameters)
{
  int b;
  char pwm_url[] = "http://192.168.43.160:3000";
  char data[] = "/upload?Nhietdo=%.2f&DO_baohoa=%.2f&NTU=%.2f&WQI_NTU=%.2f&TDS=%.2f";
  char post_data[200];
  esp_err_t err;

  esp_http_client_config_t config = {
      .url = pwm_url,
      .method = HTTP_METHOD_GET,
  };
  esp_http_client_handle_t client = esp_http_client_init(&config);
  esp_http_client_set_header(client, "Content-Type", "application/x-www-form-urlencoded");
  while (1)
  {
    xQueueReceive(queue, &b, (TickType_t)1000);
    if(b==1)
    {
      vTaskDelay(500/ portTICK_RATE_MS);
      strcpy(post_data, "");
      snprintf(post_data, sizeof(post_data), data, T, DObh, ntu, WQIdoduc, tds);
      ESP_LOGI(TAG, "post = %s", post_data);
      esp_http_client_set_post_field(client, post_data, strlen(post_data));
      esp_http_client_set_url(client, post_data);
      err = esp_http_client_perform(client);

      if (err == ESP_OK)
      {
        int status_code = esp_http_client_get_status_code(client);
        if (status_code == 200)
        {
          ESP_LOGI(TAG, "Message sent Successfully");
        }
        else
        {
          ESP_LOGI(TAG, "Message sent Failed");
          goto exit;
        }
      }
      else
      {
        ESP_LOGI(TAG, "Message sent Failed1");
        goto exit;
      }
    }
  }
  exit:
    esp_http_client_cleanup(client);
    vTaskDelete(NULL);
}
void app_main(void)
{
  esp_adc_cal_characterize(ADC_UNIT_1, ADC_ATTEN_DB_11, ADC_WIDTH_BIT_DEFAULT, 0, &adc1_chars); // hiệu chỉnh ADC1 ở mức suy giảm 11db
  adc1_config_width(ADC_WIDTH_BIT_DEFAULT);                                                     // đặt cấu hình ADC1 ở độ rộng bit mặc định (12bit)
  adc1_config_channel_atten(ADC1_CHANNEL_4, ADC_ATTEN_DB_11);                                   // đặt tham số suy giảm của ADC1 kênh 4 là GPIO32 thành 11db
  adc1_config_channel_atten(ADC1_CHANNEL_5, ADC_ATTEN_DB_11);
  adc1_config_channel_atten(ADC1_CHANNEL_6, ADC_ATTEN_DB_11);
  queue = xQueueCreate(5, sizeof(int));

  esp_err_t ret = nvs_flash_init();
  if (ret == ESP_ERR_NVS_NO_FREE_PAGES || ret == ESP_ERR_NVS_NEW_VERSION_FOUND)
  {
    ESP_ERROR_CHECK(nvs_flash_erase());
    ret = nvs_flash_init();
  }
  ESP_ERROR_CHECK(ret);
  connect_wifi();
  // r = R0 * exp(-B / T0);
  // V1 = getMedianNum(analogBuffer, SCOUNT, ADC1_CHANNEL_4);
  // R2 = R1 * (V0 / V1 - 1);
  // T = B / log(R2 / r) - 273.15;
  // printf("T=%f\n ", T);
  //TDS_calib(nvsHandle, ADC1_CHANNEL_6, 5000.0, 4096.0, "storage", 111.0, T);
  TDS_init_param(&nvsHandle);
  if (wifi_connect_status)
  {
    xTaskCreatePinnedToCore(DO_bh, "DO_bh", 2048, NULL, 10, NULL, 0);
    xTaskCreatePinnedToCore(NTU, "NTU", 2048, NULL, 10, NULL, 1);
    xTaskCreatePinnedToCore(TDS, "TDS", 2048, NULL, 10, NULL, 0);

    xTaskCreate(&send_data_to_pwm, "send_data_to_pwm", 8192, NULL, 10, NULL);
  }
}
